In [2]:
# TF-IDF(Term Frequency-Inverse Document Frequency)
# TF-IDF is SLM(Statistical Language Model), it is not considered a language model in the same sense as other techniques like neural network-based models (e.g. BERT or GPT).

import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

data = pd.read_csv('movies_metadata.csv', low_memory=False)
print(data.head(1))

# 상위 1만개의 샘플을 data에 저장
data = data.head(10000)

   adult                              belongs_to_collection    budget  \
0  False  {'id': 10194, 'name': 'Toy Story Collection', ...  30000000   

                                              genres  \
0  [{'id': 16, 'name': 'Animation'}, {'id': 35, '...   

                               homepage   id    imdb_id original_language  \
0  http://toystory.disney.com/toy-story  862  tt0114709                en   

  original_title                                           overview  ...  \
0      Toy Story  Led by Woody, Andy's toys live happily in his ...  ...   

  release_date      revenue runtime                          spoken_languages  \
0   1995-10-30  373554033.0    81.0  [{'iso_639_1': 'en', 'name': 'English'}]   

     status  tagline      title  video vote_average vote_count  
0  Released      NaN  Toy Story  False          7.7     5415.0  

[1 rows x 24 columns]


In [3]:
# overview 열에 존재하는 모든 결측값을 전부 카운트하여 출력
# 결과 : 135개의 Null 값이 있음
print('Overview 열의 결측값의 수:',data['overview'].isnull().sum())

# 결측값을 빈 값으로 대체
data['overview'] = data['overview'].fillna('')

# 결과 : 47,847차원의 문서 벡터가 10,000개 존재
tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf.fit_transform(data['overview'])
print('TF-IDF 행렬의 크기(shape) :',tfidf_matrix.shape)

Overview 열의 결측값의 수: 29
TF-IDF 행렬의 크기(shape) : (10000, 32350)


In [14]:
# 10,000개의 문서 벡터에 대해서 상호 간의 코사인 유사도 계산
# 결과 : 10,000개의 각 문서 벡터(행)에 대해
# 자기 자신(유사도값=1)을 포함한 10,000개의 다른 문서 벡터들 간의 유사도
# 모든 10,000개 문서 벡터(영화 줄거리 벡터)간의 상호 유사도를 기록(열)
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)
print('코사인 유사도 연산 결과 :',cosine_sim.shape)
print(cosine_sim[:3])

코사인 유사도 연산 결과 : (10000, 10000)
[[1.         0.01682915 0.         ... 0.         0.         0.        ]
 [0.01682915 1.         0.04871976 ... 0.         0.01200997 0.        ]
 [0.         0.04871976 1.         ... 0.         0.00735515 0.        ]]


In [29]:
# 기존 데이터프레임으로부터 영화의 타이틀을 key,
# 영화의 인덱스를 value로 하는 딕셔너리(title_to_index) 생성
title_to_index = dict(zip(data['title'], data.index))

print(title_to_index.items())

# 선택한 영화의 제목을 입력하면 코사인 유사도를 통해
# 가장 overview가 유사한 10개의 영화를 찾아내는 함수
def get_recommendations(title, cosine_sim=cosine_sim):
    # 선택한 영화의 타이틀로부터 해당 영화의 인덱스를 받아온다.
    idx = title_to_index[title]

    # 해당 영화와 모든 영화와의 유사도를 가져온다.
    sim_scores = list(enumerate(cosine_sim[idx]))

    # 유사도에 따라 영화들을 정렬한다.
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # 가장 유사한 10개의 영화를 받아온다.
    sim_scores = sim_scores[1:11]

    # 가장 유사한 10개의 영화의 인덱스를 얻는다.
    movie_indices = [idx[0] for idx in sim_scores]

    # 가장 유사한 10개의 영화의 제목을 리턴한다.
    return data['title'].iloc[movie_indices]

dict_items([('Toy Story', 0), ('Jumanji', 1), ('Grumpier Old Men', 2), ('Waiting to Exhale', 3), ('Father of the Bride Part II', 4), ('Heat', 5), ('Sabrina', 888), ('Tom and Huck', 7), ('Sudden Death', 8), ('GoldenEye', 9), ('The American President', 10), ('Dracula: Dead and Loving It', 11), ('Balto', 12), ('Nixon', 13), ('Cutthroat Island', 14), ('Casino', 15), ('Sense and Sensibility', 16), ('Four Rooms', 17), ('Ace Ventura: When Nature Calls', 18), ('Money Train', 19), ('Get Shorty', 20), ('Copycat', 21), ('Assassins', 22), ('Powder', 23), ('Leaving Las Vegas', 24), ('Othello', 2733), ('Now and Then', 26), ('Persuasion', 27), ('The City of Lost Children', 28), ('Shanghai Triad', 29), ('Dangerous Minds', 30), ('Twelve Monkeys', 31), ('Wings of Courage', 32), ('Babe', 33), ('Carrington', 34), ('Dead Man Walking', 35), ('Across the Sea of Time', 36), ('It Takes Two', 37), ('Clueless', 38), ('Cry, the Beloved Country', 39), ('Richard III', 7185), ('Dead Presidents', 41), ('Restoration',

In [31]:
# 영화 Toy Story 와 overview가 유사한 영화들 Top 10 추천
get_recommendations('Toy Story')

2997                                    Toy Story 2
8327                                      The Champ
1071                          Rebel Without a Cause
3057                                Man on the Moon
1932                                      Condorman
485                                          Malice
5797                                  Class of 1984
7254                                 Africa Screams
6944                               Rivers and Tides
7615    The First $20 Million Is Always the Hardest
Name: title, dtype: object

In [13]:
# An example code for using cosine similarity in a Keras-based recommendation system:

import numpy as np
from keras.layers import Input, Dense
from keras.models import Model
from sklearn.metrics.pairwise import cosine_similarity

# Sample user profile vector
user_profile = np.array([0.2, 0.4, 0.3, 0.1])

# Sample item feature matrix
item_features = np.array([[0.2, 0.3, 0.1, 0.2],
                          [0.3, 0.4, 0.1, 0.1],
                          [0.1, 0.2, 0.6, 0.1],
                          [0.1, 0.1, 0.1, 0.7]])

print(user_profile.shape)
print(item_features.shape)

# Create a Keras model that takes the item features as input
input_layer = Input(shape=(item_features.shape[1],))
hidden_layer = Dense(50, activation='relu')(input_layer)
output_layer = Dense(1)(hidden_layer)

model = Model(inputs=input_layer, outputs=output_layer)
model.compile(optimizer='adam', loss='mae')

# Train the model on the item features
model.fit(item_features, np.zeros(item_features.shape[0]), epochs=10)

# Use the model to make predictions for the user profile vector
user_vector = np.tile(user_profile, (item_features.shape[0], 1))
print(user_vector)
print(user_vector.shape)

item_predictions = model.predict(item_features)
print(item_predictions)
print(item_predictions.shape)

item_vector = np.tile(item_predictions, (1, user_vector.shape[1]))
print(item_vector)
print(item_vector.shape)

recommendation_scores = cosine_similarity(user_vector, item_vector)

print('코사인 유사도 연산 결과 :',recommendation_scores.shape)
print(recommendation_scores)

# Sort the items by their recommendation scores
recommendation_indices = np.argsort(recommendation_scores.flatten())[::-1]
print(recommendation_scores.flatten())

# Print the top recommended items
print(recommendation_indices)

(4,)
(4, 4)
Epoch 1/10
1/1 [==============================] - 0s 277ms/step - loss: 0.2247
Epoch 2/10
1/1 [==============================] - 0s 5ms/step - loss: 0.2115
Epoch 3/10
1/1 [==============================] - 0s 5ms/step - loss: 0.1983
Epoch 4/10
1/1 [==============================] - 0s 4ms/step - loss: 0.1851
Epoch 5/10
1/1 [==============================] - 0s 4ms/step - loss: 0.1717
Epoch 6/10
1/1 [==============================] - 0s 4ms/step - loss: 0.1583
Epoch 7/10
1/1 [==============================] - 0s 3ms/step - loss: 0.1449
Epoch 8/10
1/1 [==============================] - 0s 4ms/step - loss: 0.1315
Epoch 9/10
1/1 [==============================] - 0s 3ms/step - loss: 0.1253
Epoch 10/10
1/1 [==============================] - 0s 4ms/step - loss: 0.1195
[[0.2 0.4 0.3 0.1]
 [0.2 0.4 0.3 0.1]
 [0.2 0.4 0.3 0.1]
 [0.2 0.4 0.3 0.1]]
(4, 4)
1/1 [==============================] - 0s 40ms/step
[[ 0.0718776 ]
 [ 0.1146557 ]
 [ 0.22751938]
 [-0.04004154]]
(4, 1)
[[ 0.071877